# Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

*Задание №1.*
Решить задачи регрессии и классификации на данных в соответствии с Вашим индивидуальным вариантом (см. Лаб.работы №3, 4), используя полносвязные НС; реализовать НС посредством API Keras и фреймворка TensorFlow; оценить качество полученных моделей с помощью метрик.

*Задание №2.*
Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.


*Дополнительное Задание №3*.*
1. Самостоятельно изучить отличия работы оптимизаторов Adam и RMSProp от классического градиентного спуска.
2. Реализовать градиентный спуск с использованием указанных оптимизаторов; предусмотрите возможность использования реализованных вами оптими-заторов в Вашем персептроне.

In [15]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

## Задание №1

### Загрузка и разбиение датасетов на выборки 

In [16]:
data_regression = pd.read_csv("filteredNotes\Lab3.csv")
data_classification = pd.read_csv("filteredNotes\Lab4.csv")

In [17]:
y_regression = data_regression["price"]
X_regression = data_regression.drop(["price", "Unnamed: 0"], axis=1)
y_classification = data_classification["bomb_planted"]
X_classification = data_classification.drop(["bomb_planted", "Unnamed: 0"], axis=1)

In [18]:
data_regression

,Unnamed: 0,price,bedrooms,bathrooms,sqft_living,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,0,221900.0,3,1.00,1180.0,1,0,0,7,1180,0,1955,0,1340,5650
1,1,538000.0,3,2.25,2570.0,2,0,0,7,2170,400,1951,1991,1690,7639
2,2,180000.0,2,1.00,770.0,1,0,0,6,770,0,1933,0,2720,8062
3,3,604000.0,4,3.00,1960.0,1,0,0,7,1050,910,1965,0,1360,5000
4,4,510000.0,3,2.00,1680.0,1,0,0,8,1680,0,1987,0,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21394,21608,360000.0,3,2.50,1530.0,3,0,0,8,1530,0,2009,0,1530,1509
21395,21609,400000.0,4,2.50,2310.0,2,0,0,8,2310,0,2014,0,1830,7200
21396,21610,402101.0,2,0.75,1020.0,2,0,0,7,1020,0,2009,0,1020,2007
21397,21611,400000.0,3,2.50,1600.0,2,0,0,8,1600,0,2004,0,1410,1287


In [19]:
(
    X_regression_train,
    X_regression_test,
    y_regression_train,
    y_regression_test,
) = train_test_split(X_regression, y_regression, test_size=0.2)
(
    X_classification_train,
    X_classification_test,
    y_classification_train,
    y_classification_test,
) = train_test_split(
    X_classification, y_classification, stratify=y_classification, test_size=0.2
)

In [20]:
X_classification_train

,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive
2165,25.62,9,7,39,32,78,0,1300,3200,1,0,0,1,1
2206,54.94,14,9,228,279,199,289,8400,8650,1,3,1,3,3
61083,114.94,4,1,500,500,492,300,10350,9200,4,1,2,5,5
52074,175.00,8,6,500,500,461,300,14350,10300,5,3,4,5,5
27353,14.91,4,5,371,300,400,300,3200,5000,2,3,2,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9311,169.91,6,8,500,500,0,91,10700,20450,0,0,0,5,5
55587,54.95,4,9,327,239,189,281,9050,23250,0,3,0,4,3
3409,174.94,6,6,500,500,300,0,20700,33150,2,0,2,5,5
42519,74.94,5,14,212,345,300,319,3050,20150,3,4,3,3,4


### Обучение модели регрессии

In [21]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(13,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [22]:
model_regression.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                896       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 3521 (13.75 KB)
Trainable params: 3521 (

In [23]:
model_regression.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse"
)

In [24]:
model_regression.fit(X_regression_train, y_regression_train, epochs=30)

Epoch 1/30


535/535 [==============================] - 1s 2ms/step - loss: 91628183552.0000
Epoch 2/30
535/535 [==============================] - 1s 1ms/step - loss: 63421014016.0000
Epoch 3/30
535/535 [==============================] - 1s 1ms/step - loss: 62706069504.0000
Epoch 4/30
535/535 [==============================] - 1s 1ms/step - loss: 62648619008.0000
Epoch 5/30
535/535 [==============================] - 1s 1ms/step - loss: 61953138688.0000
Epoch 6/30
535/535 [==============================] - 1s 1ms/step - loss: 60821696512.0000
Epoch 7/30
535/535 [==============================] - 1s 1ms/step - loss: 58916417536.0000
Epoch 8/30
535/535 [==============================] - 1s 1ms/step - loss: 58098135040.0000
Epoch 9/30
535/535 [==============================] - 1s 1ms/step - loss: 57107570688.0000
Epoch 10/30
535/535 [==============================] - 1s 1ms/step - loss: 56689700864.0000
Epoch 11/30
535/535 [==============================] - 1s 1ms/step - loss: 55350362112.0000
Epoch 12

In [25]:
print(
    mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test))
)
print(
    mean_squared_error(y_regression_test, model_regression.predict(X_regression_test))
)

134/134 [==============================] - 0s 863us/step
144454.7277635806
134/134 [==============================] - 0s 883us/step
43509342631.446205


отработало плохо

### Обучение модели классификации

In [26]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse"
)
# verbose=None - не будет логов
model_classification_1.fit(
    X_classification_train, y_classification_train, epochs=25, verbose=None
)

In [27]:
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     19752
           1       0.12      1.00      0.21      2646

    accuracy                           0.12     22398
   macro avg       0.06      0.50      0.11     22398
weighted avg       0.01      0.12      0.02     22398

[[    0 19752]
 [    0  2646]]


c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

результаты не очень - очень много ошибок второго рода

In [28]:
w0 = 1 / y_classification_train[y_classification_train == 0].shape[0]
w1 = 1 / y_classification_train[y_classification_train == 1].shape[0]

In [29]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy"
)
model_classification_1.fit(
    X_classification_train,
    y_classification_train,
    epochs=25,
    verbose=None,
    class_weight={0: w0, 1: w1},
)
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.90      0.95     19752
           1       0.58      0.98      0.73      2646

    accuracy                           0.91     22398
   macro avg       0.79      0.94      0.84     22398
weighted avg       0.95      0.91      0.92     22398

[[17845  1907]
 [   57  2589]]


Стало гораздно лучше

In [30]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 2 нейрона и softmax
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)
# в качестве функции активации используется категориальная кроссэнтропия
# используем разряженный (sparse) вариант, поскольку значения целевого признака не закодированы One-Hot кодированием
model_classification_2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
    loss="sparse_categorical_crossentropy",
)
model_classification_2.fit(
    X_classification_train,
    y_classification_train,
    epochs=25,
    verbose=None,
    class_weight={0: w0, 1: w1},
)

In [31]:
model_classification_2.predict(X_classification_test, verbose=None)[:5]

array([[0.3844817 , 0.6155183 ],
       [0.9565685 , 0.04343155],
       [0.19388534, 0.8061147 ],
       [0.36006734, 0.6399327 ],
       [0.9863929 , 0.01360706]], dtype=float32)

In [32]:
y_pred = [
    np.argmax(pred)
    for pred in model_classification_2.predict(X_classification_test, verbose=None)
]

In [33]:
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.70      0.82     19752
           1       0.30      0.97      0.46      2646

    accuracy                           0.73     22398
   macro avg       0.65      0.83      0.64     22398
weighted avg       0.91      0.73      0.78     22398

[[13760  5992]
 [   92  2554]]


In [34]:
import pickle
    
model_classification_1.save('C:/MLforGIt/ML_RGR/Models/NN.h5')

c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Тут напротив - большой завал в один из классов

Попробуем устранить дисбаланс с помощью NearMiss

In [35]:
print(f"1: {sum(y_classification_train == 1)}")
print(f"0: {sum(y_classification_train == 0)}")

1: 10582
0: 79008


In [36]:
nm = NearMiss()
X_classification_train, y_classification_train = nm.fit_resample(
    X_classification_train, y_classification_train.ravel()
)

In [37]:
print(f"1: {sum(y_classification_train == 1)}")
print(f"0: {sum(y_classification_train == 0)}")

1: 10582
0: 10582


In [38]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse"
)
# verbose=None - не будет логов
model_classification_1.fit(
    X_classification_train, y_classification_train, epochs=25, verbose=None
)

In [39]:
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94     19752
           1       0.00      0.00      0.00      2646

    accuracy                           0.88     22398
   macro avg       0.44      0.50      0.47     22398
weighted avg       0.78      0.88      0.83     22398

[[19752     0]
 [ 2646     0]]


c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

с binary_crossentropy получилось лучше всего

In [40]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(14,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy"
)
model_classification_1.fit(
    X_classification_train,
    y_classification_train,
    epochs=25,
    verbose=None,
    class_weight={0: w0, 1: w1},
)
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.47      0.64     19752
           1       0.20      1.00      0.34      2646

    accuracy                           0.54     22398
   macro avg       0.60      0.74      0.49     22398
weighted avg       0.91      0.54      0.61     22398

[[ 9350 10402]
 [    0  2646]]


In [41]:
model_classification_1.save('C:/MLforGIt/ML_RGR/Models/NN.h5')

c:\Users\vlszg\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
